In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET

In [ ]:
# Imports 
from utils import get_pickle, get_order_pf
import asyncio
import pandas as pd
import numpy as np

# Build `states`

In [ ]:
# imports
from utils import qualify_conIds, get_dte
from utils import clean_ib_util_df
from ib_insync import MarketOrder

In [ ]:
# get unds, open orders and portfolio
unds = set(get_pickle(DATAPATH / 'unds.pkl').keys())
df_openorder, df_pf = asyncio.run(get_order_pf(PORT))

In [ ]:
df_pf

In [ ]:
# get margins of all positions
CID = _vars.CID
desc = "Qualifiying Reap Contracts"
conId_list = df_pf.conId.to_list()

# get margins of all positions in the portfolio
pf_contracts = asyncio.run(qualify_conIds(PORT, conId_list, f"Qualifying {MARKET} conIds"))

In [ ]:
# integrate df_pf with multiplier

df1 = clean_ib_util_df(pf_contracts).set_index('conId')
df2 = df_pf.set_index('conId')

cols_to_use = df2.columns.difference(df1.columns)
df_pf = df1.join(df2[cols_to_use])

In [ ]:
# Get DTEs
df_pf.insert(4, 'dte', df_pf.expiry.apply(lambda x: get_dte(x, MARKET)))
df_pf.loc[df_pf.dte <=0, "dte"] = 0

# Get the costPrice
df_pf.insert(9, 'costPrice', abs(df_pf.avgCost/df_pf.position))

# Assign the actions
df_pf = df_pf.assign(action=np.where(df_pf.position < 0, "BUY", "SELL"))

In [ ]:
# join the multiplier
s = pd.to_numeric(df_pf.multiplier)
s.fillna(1, inplace=True)
df_pf = df_pf.assign(multiplier=s)

In [ ]:
# build the order
wif_order = [MarketOrder(action, totalQuantity) 
             for action, totalQuantity 
             in zip(df_pf.action, abs(df_pf.position*df_pf.multiplier))]
df_pf = df_pf.assign(wif_order = wif_order)

In [ ]:
df_pf

## Unsowed

In [ ]:
# No orders to sow and no existing positions

options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'

long = df_pf.position > 0
long_options = long & options
df_pf[long_options]

unsowed = unds - set(df_openorder.symbol) - set(df_pf.symbol)
unsowed

# Orphaned

In [ ]:
# Long calls or puts without any underlying stock position
options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'
long = df_pf.position >= 1
long_stocks = long & stocks
long_options = long & options
orphaned = long_stocks & long_options

orphaned = df_pf[orphaned]
orphaned

## Unreaped

In [ ]:
# Open position without a reap order
df_reap = df_pf[options]

In [ ]:
from utils import get_margins
import asyncio

In [ ]:
from utils import get_pickle
df_nakeds = get_pickle(DATAPATH / 'df_naked_targets.pkl')[OPT_COLS]

In [ ]:
from ib_insync import MarketOrder
from loguru import logger
from utils import clean_a_margin
from tqdm import tqdm
BAR_FORMAT = _vars.BAR_FORMAT
import pandas as pd
from utils import chunk_me

async def get_a_margin(ib: IB, 
                       contract,
                       lot_path: Path=None,
                       ACTION: str='SELL'):
    
    """Gets a margin"""
    lot_size = 1 # Default for SNP

    if lot_path: # For NSE
        lot_size = get_pickle(lot_path).get(contract.symbol, None)

    order = MarketOrder(ACTION, lot_size)

    def onError(reqId, errorCode, errorString, contract):
        logger.error(f"{contract.localSymbol} with reqId: {reqId} has errorCode: {errorCode} error: {errorString}")

    ib.errorEvent += onError
    wif = await ib.whatIfOrderAsync(contract, order)
    ib.errorEvent -= onError
    logger.remove()

    try:
        output = clean_a_margin(wif, contract.conId)
    except KeyError:
        output = {contract.conId: {
                  'margin': None,
                  'comm': None}}
        
        logger.error(f"{contract.localSymbol} has no margin and commission")

    output[contract.conId]['lot_size'] = lot_size
    return output



async def get_margins(port: int, 
                      contracts: list, 
                      lot_path: Path=None,
                      ACTION: str='SELL', 
                      chunk_size: int=100,
                      CID: int=0):

    # contracts = df_cts.qualified_opts.to_list()

    results = list()

    pbar = tqdm(total=len(contracts),
                    desc="Getting margins:",
                    bar_format = BAR_FORMAT,
                    ncols=80,
                    leave=True,
                )

    # df_cts = df_cts.assign(conId=[c.conId for c in contracts]).\
    #             set_index('conId')
    df_cts = pd.DataFrame({"contract": contracts, 
                           "conId": [c.conId for c in contracts]}).set_index('conId')

    chunks = chunk_me(contracts, chunk_size)

    with await IB().connectAsync(port=port, clientId=CID) as ib:
        
        for cts in chunks:

            tasks = [asyncio.create_task(get_a_margin(ib, contract, lot_path, ACTION), name= contract.localSymbol) 
                    for contract in cts]

            margin = await asyncio.gather(*tasks)

            results += margin
            pbar.update(len(cts))
            pbar.refresh()

    flat_results ={k: v for r in results for k, v in r.items()}
    df_mgncomm = pd.DataFrame(flat_results).T
    df_out = df_cts.join(df_mgncomm).reset_index()

    pbar.close()

    return df_out

In [ ]:
contracts = pf_contracts

In [ ]:
df = asyncio.run(get_margins(PORT, contracts, None, 'BUY'))